**Imports**

In [1]:
import os
import dotenv
dotenv.load_dotenv(".env")

True

**Create Client**

In [2]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

**Knowledge Tools**

In [4]:
from azure.ai.projects.models import BingGroundingTool

bing_connection = project_client.connections.get(
    connection_name=os.environ["BING_CONNECTION_NAME"]
)
conn_id = bing_connection.id

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

**Create Agent**

In [5]:
from azure.ai.projects.models import FunctionTool

agent = project_client.agents.create_agent(
    name="my-knowledge-tool-agent",
    model=os.getenv("chatModel"),
    instructions="You are a helpful assistant",
    tools=bing.definitions,
    # headers={"x-ms-enable-preview": "true"},
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_n5szxwAtwUMZ0FLRCXqrq2Ef


**Create Thread & Adding Message**

In [6]:
user_message = "hi, what's the weather in New York?"
# user_message = "Do a web search for latest news in AI"
user_message = "What is the latest news in AI when it comes to Google?"

In [7]:
# with project_client:
thread = project_client.agents.create_thread()
agent = project_client.agents.get_agent(agent_id=agent.id)

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=user_message,
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_2nSh9ED4XLSsiCCbo4JgRIGD


**Running Agent with Thread**

In [8]:
from azure.ai.projects.models import MessageRole

# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished with status: completed


In [9]:
# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)

for m in reversed(messages.data):
    if m.content[0].text.annotations:
        print(f"Message ({m.role}): {m.content[0].text.value}")
        for ref in m.content[0].text.annotations:
            print(f"URL Citation: [{ref.url_citation.title}]({ref.url_citation.url})")
    else:
        print(f"Message ({m.role}): {m.content[0].text.value}")

Message (user): What is the latest news in AI when it comes to Google?
Message (assistant): Recently, Google has been involved in several notable AI advancements. Papa Johns is now using Google Cloud's AI tools for delivery route optimization, customer-facing chatbots, and personalized rewards【3†source】. Additionally, at the upcoming Google Cloud Next 2025 event, Google will showcase its latest work in AI, including Gemini-powered databases for developer innovation, AI-assisted code generation, and sustainable AI infrastructure【3†source】【7†source】. This emphasizes Google's commitment to integrating AI across various industries and enhancing developer productivity and sustainability.
URL Citation: [Alibaba to Launch Qwen 3 AI Model in April Amid Rising AI Competition](https://www.outlookbusiness.com/start-up/news/alibaba-to-launch-qwen-3-ai-model-in-april-amid-rising-ai-competition)
URL Citation: [Alibaba to Launch Qwen 3 AI Model in April Amid Rising AI Competition](https://www.outlook

**Delete Agent & Thread**

In [10]:
# Delete the agent when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

# Delete Thread when done
project_client.agents.delete_thread(thread_id=thread.id)
print(f"thread: {thread.id} deleted")

Deleted agent
thread: thread_1P1CnwQGcWPXo9PHh48F1cJx deleted
